<a href="https://colab.research.google.com/github/AlviJohn/AmazonRecoEngine/blob/main/Text_and_Image_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from os import listdir
import os
import gzip
import json
from google.colab import drive
import gc
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display

## Amazon Dataset --https://nijianmo.github.io/amazon/index.html
###Mounting Google Drive

CPU times: user 242 µs, sys: 0 ns, total: 242 µs
Wall time: 246 µs


In [ ]:
####Mounting the Drive
drive.mount('/content/drive')
path ="/content/drive/MyDrive/LJMU Research Topic/colab_experiments/data"
RAWDATA_PATH = path + "/raw"
EMBEDDINGSDATA_PATH = path + "/embeddings"
os.chdir(RAWDATA_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***Text Embeddings***

In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00


In [ ]:
###Reading the product metadata
meta_data = pd.read_csv(RAWDATA_PATH + "/meta_data_subset_pantry.csv")
meta_data=meta_data[~ meta_data['description'].isna()]
meta_data['description'] = meta_data['description'].apply(lambda x: x[:510])




In [ ]:
###Downloading the BERT Tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
%%time
max_len = 500
embeddings_dict= {}
model.eval()
for index, row in meta_data.iterrows():
  tokenized_text = tokenizer.encode(row['description'], add_special_tokens=True)
  padded_seq = torch.zeros(max_len)
  padded_seq[:len(tokenized_text)] = torch.tensor(tokenized_text)
  padded_seq = padded_seq.to(torch.int64)
  with torch.no_grad():
    embedding = model(padded_seq.unsqueeze(0))[0].squeeze(0).mean(dim=0)
    embeddings_dict[row['asin']]=embedding
    
embeddings_path = os.path.join(EMBEDDINGSDATA_PATH, "productdescription_embeddings_pantry.pt")
torch.save(embeddings_dict, embeddings_path)


CPU times: user 2h 23min 46s, sys: 8.47 s, total: 2h 23min 55s
Wall time: 2h 24min 36s



**Image Embeddings**

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import os

In [ ]:
%%time
#Load pre-trained VGG-16 model
vgg16 = models.vgg16(pretrained=True)
# Remove the last fully connected layer
vgg16 = torch.nn.Sequential(*list(vgg16.children())[:-1])
vgg16.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 219MB/s]


CPU times: user 2.06 s, sys: 1.81 s, total: 3.87 s
Wall time: 4.36 s


Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
# Define the directory path of the images
image_dir = RAWDATA_PATH + "/images_pantry"
# Get the list of image filenames in the directory
image_filenames = os.listdir(image_dir)

In [ ]:
%%time
# Create an empty list to store the embeddings and image names
embeddings = []
image_names = []
embeddings_dict={}
# Loop through each image and generate its VGG16 embedding
for filename in image_filenames:
    # Load the image and apply the transformations
    image_path = os.path.join(image_dir, filename)
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    
    # Generate the VGG16 embedding for the image
    with torch.no_grad():
        embedding = vgg16(image)
    
    # Add the embedding and image name to the lists
    embeddings = embedding.squeeze(0).flatten()
    image_names = filename.rsplit('.', maxsplit=1)[0]
    # Save the tensor and image names to a dictionary
    embeddings_dict[image_names]=embeddings

embeddings_path = os.path.join(EMBEDDINGSDATA_PATH, "productimage_embeddings_pantry.pt")
torch.save(embeddings_dict, embeddings_path)

CPU times: user 42min 5s, sys: 1min 45s, total: 43min 51s
Wall time: 1h 2min 10s


Product Embeddings